In [1]:
import dgl
import torch
import pickle as pkl
import sys
import networkx as nx
import numpy as np
import scipy.sparse as sp
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv
import time
from sklearn.metrics import average_precision_score, roc_auc_score

In [2]:
# check device
device = torch.device(
    "cuda:{}".format(0) if torch.cuda.is_available() else "cpu"
)

In [3]:
def create_fc_matrices(scan, window_size=30, step_size=30):
    """
    Create functional connectivity matrices using a sliding window approach.
    """
    n_timepoints = scan.shape[1]
    fc_matrices = []

    for start in range(0, n_timepoints - window_size + 1, step_size):
        window = scan[:, start:start + window_size]
        correlation_matrix = np.corrcoef(window)
        fc_matrices.append(correlation_matrix)

    return fc_matrices


def threshold_fc_matrix(fc_matrix, percentile=5):
    """
    Threshold the FC matrix to keep only the top percentile connections.
    """
    threshold = np.percentile(fc_matrix[np.tril_indices_from(fc_matrix, k=-1)], 100 - percentile)   
    graph = (fc_matrix > threshold).astype(int)
    np.fill_diagonal(graph, 0)  # remove self-edges
    return graph


def create_networkx_graph(matrix):
    G = nx.Graph(matrix)
    return G


def convert_to_pyg_graph(nx_graph):
    graph = nx.Graph(nx_graph)
    edges = torch.tensor(list(graph.edges), dtype=torch.long).t().contiguous()
    x = torch.tensor(np.identity(graph.number_of_nodes()), dtype=torch.float)
    
    # For unsupervised learning, you typically don't have node labels
    # If you have node features, replace x with them
    return Data(x=x, edge_index=edges)

In [4]:
def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = (
        adj_.dot(degree_mat_inv_sqrt)
        .transpose()
        .dot(degree_mat_inv_sqrt)
        .tocoo()
    )
    return adj_normalized, sparse_to_tuple(adj_normalized)

def load_data(dataset):
    # load the data: x, tx, allx, graph
    names = ["x", "tx", "allx", "graph"]
    data_path = "/home/matteoc/graphs-nn/data/"
    objects = []
    for i in range(len(names)):
        with open(data_path+"data_cora/ind.{}.{}".format(dataset, names[i]), "rb") as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding="latin1"))
            else:
                objects.append(pkl.load(f))
    x, tx, allx, graph = tuple(objects)
    test_idx_reorder = parse_index_file(
        data_path+"data_cora/ind.{}.test.index".format(dataset)
    )
    test_idx_range = np.sort(test_idx_reorder)

    if dataset == "citeseer":
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(
            min(test_idx_reorder), max(test_idx_reorder) + 1
        )
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range - min(test_idx_range), :] = tx
        tx = tx_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    return adj, features


def mask_test_edges(adj):
    # Function to build test set with 10% positive links
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
    # TODO: Clean up.

    # Remove diagonal elements
    adj = adj - sp.dia_matrix(
        (adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape
    )
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 10.0))
    num_val = int(np.floor(edges.shape[0] / 20.0))

    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val : (num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(
        edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0
    )

    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges):
            continue
        if ismember([idx_j, idx_i], train_edges):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])

    assert ~ismember(test_edges_false, edges_all)
    assert ~ismember(val_edges_false, edges_all)
    assert ~ismember(val_edges, train_edges)
    assert ~ismember(test_edges, train_edges)
    assert ~ismember(val_edges, test_edges)

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = sp.csr_matrix(
        (data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape
    )
    adj_train = adj_train + adj_train.T

    # NOTE: these edge lists only contain single direction of edge!
    return (
        adj_train,
        train_edges,
        val_edges,
        val_edges_false,
        test_edges,
        test_edges_false,
    )

In [5]:
class VGAEModel(nn.Module):
    def __init__(self, in_dim, hidden1_dim, hidden2_dim):
        super(VGAEModel, self).__init__()
        self.in_dim = in_dim
        self.hidden1_dim = hidden1_dim
        self.hidden2_dim = hidden2_dim

        layers = [
            GraphConv(
                self.in_dim,
                self.hidden1_dim,
                activation=F.relu,
                allow_zero_in_degree=True,
            ),
            GraphConv(
                self.hidden1_dim,
                self.hidden2_dim,
                activation=lambda x: x,
                allow_zero_in_degree=True,
            ),
            GraphConv(
                self.hidden1_dim,
                self.hidden2_dim,
                activation=lambda x: x,
                allow_zero_in_degree=True,
            ),
        ]
        self.layers = nn.ModuleList(layers)

    def encoder(self, g, features):
        h = self.layers[0](g, features)
        self.mean = self.layers[1](g, h)
        self.log_std = self.layers[2](g, h)
        gaussian_noise = torch.randn(features.size(0), self.hidden2_dim).to(
            device
        )
        sampled_z = self.mean + gaussian_noise * torch.exp(self.log_std).to(
            device
        )
        return sampled_z

    def decoder(self, z):
        adj_rec = torch.sigmoid(torch.matmul(z, z.t()))
        return adj_rec

    def forward(self, g, features):
        z = self.encoder(g, features)
        adj_rec = self.decoder(z)
        return adj_rec

In [6]:
path_conn = '../data/hcp/raw/100206_0.npy'
path_brain_region = '../data/brain_regions.csv'

# Load the connectivity matrix
scan = np.load(path_conn)[:, :490]
print('CONNECTIVITY_MATRIX_SHAPE: ', scan.shape)

fc_matrices = create_fc_matrices(scan, 490, 1)
print('FC_MATRIX_SHAPE:', np.array(fc_matrices).shape)

graphs = [threshold_fc_matrix(fc) for fc in fc_matrices]
print("GRAPHS_SHAPE:" ,np.array(graphs).shape)

# fc_matrices = np.array(fc_matrices)[0]
fc_matrices_tr = sp.csr_matrix(graphs[0])
fc_matrices_last = fc_matrices_tr.astype(int)
print("FC_MATRIX_SHAPE_NEW:" , fc_matrices_last.shape)



CONNECTIVITY_MATRIX_SHAPE:  (360, 490)
FC_MATRIX_SHAPE: (1, 360, 360)
GRAPHS_SHAPE: (1, 360, 360)
FC_MATRIX_SHAPE_NEW: (360, 360)


In [7]:
adj_orig = fc_matrices_last
print(type(fc_matrices_last))

<class 'scipy.sparse._csr.csr_matrix'>


In [8]:
print(sparse_to_tuple(fc_matrices_last))
print(sparse_to_tuple(fc_matrices_last)[0].shape)

(array([[  0,   3],
       [  0,   4],
       [  0,   5],
       ...,
       [359, 264],
       [359, 349],
       [359, 358]], dtype=int32), array([1, 1, 1, ..., 1, 1, 1]), (360, 360))
(6462, 2)


In [9]:
(
    adj_train,
    train_edges,
    val_edges,
    val_edges_false,
    test_edges,
    test_edges_false,
) = mask_test_edges(fc_matrices_last)
adj = adj_train        # remember

In [63]:
print(sparse_to_tuple(adj_train)[0].shape) # --> 5494 = 2747*2 (nodi) 
print(train_edges.shape)
print(val_edges.shape)
print(test_edges.shape)
print(len(val_edges_false))
print(len(test_edges_false))

(5494, 2)
(2747, 2)
(161, 2)
(323, 2)
161
323


In [10]:
# Some preprocessing
adj_normalization, adj_norm = preprocess_graph(adj)

# Create model
graph = dgl.from_scipy(adj_normalization)
graph.remove_self_loop()
pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
norm = (
    adj.shape[0]
    * adj.shape[0]
    / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)
)

adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)

In [11]:
graph

Graph(num_nodes=360, num_edges=5854,
      ndata_schemes={}
      edata_schemes={})

In [15]:
adj_label[0].shape       # --> 5854 = 5494 + 360 (self loops)

(5854, 2)

In [16]:
adj_norm = torch.sparse.FloatTensor(       # think never used 
    torch.LongTensor(adj_norm[0].T),
    torch.FloatTensor(adj_norm[1]),
    torch.Size(adj_norm[2]),
)
adj_label = torch.sparse.FloatTensor(
    torch.LongTensor(adj_label[0].T),
    torch.FloatTensor(adj_label[1]),
    torch.Size(adj_label[2]),
)
features = sparse_to_tuple(fc_matrices_last.tocoo())   # --> (6462, 2)
features = torch.sparse.FloatTensor(
    torch.LongTensor(features[0].T),
    torch.FloatTensor(features[1]),
    torch.Size(features[2]),
)

weight_mask = adj_label.to_dense().view(-1) == 1
weight_tensor = torch.ones(weight_mask.size(0))
weight_tensor[weight_mask] = pos_weight

features = features.to_dense()
print(features.shape)
in_dim = features.shape[-1]

torch.Size([360, 360])


In [17]:
features

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.]])

In [18]:
vgae_model = VGAEModel(in_dim, 32, 16)
# create training component
optimizer = torch.optim.Adam(vgae_model.parameters(), lr=0.01)
print(
    "Total Parameters:",
    sum([p.nelement() for p in vgae_model.parameters()]),
)

Total Parameters: 12608


In [19]:
import time
from sklearn.metrics import average_precision_score, roc_auc_score


def get_scores(edges_pos, edges_neg, adj_rec):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    preds = []
    pos = []
    for e in edges_pos:
        # print(e)
        # print(adj_rec[e[0], e[1]])
        preds.append(sigmoid(adj_rec[e[0], e[1]].item()))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]].data))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

def get_acc(adj_rec, adj_label):
    labels_all = adj_label.to_dense().view(-1).long()
    preds_all = (adj_rec > 0.5).view(-1).long()
    accuracy = (preds_all == labels_all).sum().float() / labels_all.size(0)
    return accuracy, labels_all, preds_all

for epoch in range(200):
    t = time.time()

    # Training and validation using a full graph
    vgae_model.train()

    logits = vgae_model.forward(graph, features)

    # compute loss
    loss = norm * F.binary_cross_entropy(
        logits.view(-1), adj_label.to_dense().view(-1), weight=weight_tensor
    )
    kl_divergence = (
        0.5
        / logits.size(0)
        * (
            1
            + 2 * vgae_model.log_std
            - vgae_model.mean**2
            - torch.exp(vgae_model.log_std) ** 2
        )
        .sum(1)
        .mean()
    )
    loss -= kl_divergence

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_acc, true_all, preds_all = get_acc(logits, adj_label)

    val_roc, val_ap = get_scores(val_edges, val_edges_false, logits)

    # Print out performance
    print(
        "Epoch:",
        "%04d" % (epoch + 1),
        "train_loss=",
        "{:.5f}".format(loss.item()),
        "train_acc=",
        "{:.5f}".format(train_acc),
        "val_roc=",
        "{:.5f}".format(val_roc),
        "val_ap=",
        "{:.5f}".format(val_ap),
        "time=",
        "{:.5f}".format(time.time() - t),
    )

test_roc, test_ap = get_scores(test_edges, test_edges_false, logits)
print(
    "End of training!",
    "test_roc=",
    "{:.5f}".format(test_roc),
    "test_ap=",
    "{:.5f}".format(test_ap),
)

Epoch: 0001 train_loss= 1.74276 train_acc= 0.50020 val_roc= 0.47537 val_ap= 0.50637 time= 0.06780
Epoch: 0002 train_loss= 1.43413 train_acc= 0.48815 val_roc= 0.47691 val_ap= 0.48982 time= 0.04073


/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage 

Epoch: 0003 train_loss= 1.19284 train_acc= 0.41005 val_roc= 0.64519 val_ap= 0.62243 time= 0.03650
Epoch: 0004 train_loss= 1.09450 train_acc= 0.38585 val_roc= 0.71672 val_ap= 0.69726 time= 0.02981
Epoch: 0005 train_loss= 1.04455 train_acc= 0.34367 val_roc= 0.72042 val_ap= 0.70504 time= 0.02600
Epoch: 0006 train_loss= 0.94131 train_acc= 0.36625 val_roc= 0.77030 val_ap= 0.74616 time= 0.02492
Epoch: 0007 train_loss= 0.84473 train_acc= 0.38779 val_roc= 0.72200 val_ap= 0.64482 time= 0.02413
Epoch: 0008 train_loss= 0.83090 train_acc= 0.37301 val_roc= 0.63458 val_ap= 0.57646 time= 0.02458
Epoch: 0009 train_loss= 0.75811 train_acc= 0.41574 val_roc= 0.70522 val_ap= 0.60810 time= 0.02367
Epoch: 0010 train_loss= 0.72408 train_acc= 0.42213 val_roc= 0.69986 val_ap= 0.61617 time= 0.02376
Epoch: 0011 train_loss= 0.72251 train_acc= 0.40196 val_roc= 0.75483 val_ap= 0.66141 time= 0.02319


/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage 

Epoch: 0012 train_loss= 0.68465 train_acc= 0.43065 val_roc= 0.77115 val_ap= 0.70391 time= 0.02355
Epoch: 0013 train_loss= 0.64971 train_acc= 0.46176 val_roc= 0.81752 val_ap= 0.76515 time= 0.02302
Epoch: 0014 train_loss= 0.61958 train_acc= 0.50318 val_roc= 0.81008 val_ap= 0.77239 time= 0.02203
Epoch: 0015 train_loss= 0.60351 train_acc= 0.52537 val_roc= 0.84025 val_ap= 0.80602 time= 0.02273
Epoch: 0016 train_loss= 0.59404 train_acc= 0.53113 val_roc= 0.86825 val_ap= 0.85213 time= 0.02218
Epoch: 0017 train_loss= 0.60079 train_acc= 0.51975 val_roc= 0.86088 val_ap= 0.83666 time= 0.02250
Epoch: 0018 train_loss= 0.60594 train_acc= 0.51032 val_roc= 0.89877 val_ap= 0.90314 time= 0.02218
Epoch: 0019 train_loss= 0.59436 train_acc= 0.53150 val_roc= 0.87639 val_ap= 0.86395 time= 0.02297
Epoch: 0020 train_loss= 0.59816 train_acc= 0.53134 val_roc= 0.88534 val_ap= 0.87026 time= 0.02196


...

Epoch: 0182 train_loss= 0.47391 train_acc= 0.59461 val_roc= 0.95220 val_ap= 0.94776 time= 0.02445
Epoch: 0183 train_loss= 0.47349 train_acc= 0.59481 val_roc= 0.94487 val_ap= 0.94173 time= 0.02406
Epoch: 0184 train_loss= 0.47335 train_acc= 0.59583 val_roc= 0.95915 val_ap= 0.95757 time= 0.02586
Epoch: 0185 train_loss= 0.47327 train_acc= 0.59202 val_roc= 0.95691 val_ap= 0.96394 time= 0.02431
Epoch: 0186 train_loss= 0.47323 train_acc= 0.59495 val_roc= 0.95525 val_ap= 0.94855 time= 0.02364
Epoch: 0187 train_loss= 0.47353 train_acc= 0.59642 val_roc= 0.95239 val_ap= 0.94496 time= 0.02529
Epoch: 0188 train_loss= 0.47323 train_acc= 0.59511 val_roc= 0.95826 val_ap= 0.96105 time= 0.02454
Epoch: 0189 train_loss= 0.47307 train_acc= 0.60215 val_roc= 0.95691 val_ap= 0.95432 time= 0.02417
Epoch: 0190 train_loss= 0.47373 train_acc= 0.59785 val_roc= 0.95961 val_ap= 0.95766 time= 0.02454


/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage 

Epoch: 0191 train_loss= 0.47302 train_acc= 0.59657 val_roc= 0.95143 val_ap= 0.95424 time= 0.02399
Epoch: 0192 train_loss= 0.47218 train_acc= 0.59486 val_roc= 0.94472 val_ap= 0.93390 time= 0.02464
Epoch: 0193 train_loss= 0.46990 train_acc= 0.59853 val_roc= 0.95556 val_ap= 0.95285 time= 0.02530
Epoch: 0194 train_loss= 0.47258 train_acc= 0.59954 val_roc= 0.94884 val_ap= 0.94771 time= 0.02464
Epoch: 0195 train_loss= 0.47204 train_acc= 0.59688 val_roc= 0.95949 val_ap= 0.95862 time= 0.02417
Epoch: 0196 train_loss= 0.47292 train_acc= 0.59636 val_roc= 0.96551 val_ap= 0.96913 time= 0.02498
Epoch: 0197 train_loss= 0.47134 train_acc= 0.59645 val_roc= 0.96131 val_ap= 0.96390 time= 0.02421
Epoch: 0198 train_loss= 0.47284 train_acc= 0.59934 val_roc= 0.95039 val_ap= 0.94749 time= 0.02460
Epoch: 0199 train_loss= 0.46975 train_acc= 0.59989 val_roc= 0.95779 val_ap= 0.95720 time= 0.02420


/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
/home/matteoc/miniconda3/envs/dbgdgm/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage 

Epoch: 0200 train_loss= 0.47147 train_acc= 0.59633 val_roc= 0.96192 val_ap= 0.96355 time= 0.02355
End of training! test_roc= 0.96718 test_ap= 0.96686
